In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.filters import frangi
from skimage.morphology import (
    remove_small_objects,
    binary_closing,
    binary_opening,
    disk
)

# ---------------- CONFIG ----------------
ROI_DIR = r"roi"
OUT_MASK_DIR = r"masks_improved"
MIN_COMPONENT_SIZE = 150
# ----------------------------------------

os.makedirs(OUT_MASK_DIR, exist_ok=True)

def generate_better_mask(roi_gray):
    # ---- 1. Contrast enhancement ----
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img = clahe.apply(roi_gray)

    # ---- 2. Denoise ----
    img = cv2.GaussianBlur(img, (5,5), 0)

    # ---- 3. Vessel enhancement (FIXED Frangi) ----
    vessel = frangi(
        img,
        sigmas=range(1, 4),  # thin vessels
        alpha=0.5,
        beta=15,
        gamma=15
    )

    vessel = np.nan_to_num(vessel)
    vessel = (vessel / (vessel.max() + 1e-6) * 255).astype(np.uint8)

    # ---- 4. Adaptive threshold ----
    binary = cv2.adaptiveThreshold(
        vessel,
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        21,
        -2
    )

    # ---- 5. Morphological cleanup ----
    binary = binary_closing(binary > 0, disk(2))
    binary = binary_opening(binary, disk(1))

    # ---- 6. Remove tiny junk ----
    binary = remove_small_objects(binary, min_size=MIN_COMPONENT_SIZE)

    return (binary * 255).astype(np.uint8)

def extract_main_vessel(mask):
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask)

    if num_labels <= 1:
        return mask

    largest = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    cleaned = np.zeros_like(mask)
    cleaned[labels == largest] = 255
    return cleaned

def thicken_mask(mask):
    kernel = np.ones((3,3), np.uint8)
    return cv2.dilate(mask, kernel, iterations=1)

# -------- Batch processing --------

roi_files = [f for f in os.listdir(ROI_DIR)
             if f.lower().endswith((".png", ".jpg", ".jpeg"))]

for fname in tqdm(roi_files):
    roi_path = os.path.join(ROI_DIR, fname)
    roi_gray = cv2.imread(roi_path, cv2.IMREAD_GRAYSCALE)

    if roi_gray is None:
        continue

    mask = generate_better_mask(roi_gray)
    mask = extract_main_vessel(mask)
    mask = thicken_mask(mask)

    out_name = os.path.splitext(fname)[0] + "_mask.png"
    out_path = os.path.join(OUT_MASK_DIR, out_name)

    cv2.imwrite(out_path, mask)

print(f"✅ Improved masks saved to: {OUT_MASK_DIR}")


  0%|          | 0/1233 [00:00<?, ?it/s]C:\Users\Goyam Jain\AppData\Local\Temp\ipykernel_9492\554045562.py:52: FutureWarning: `binary_closing` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.closing` instead.
  binary = binary_closing(binary > 0, disk(2))
C:\Users\Goyam Jain\AppData\Local\Temp\ipykernel_9492\554045562.py:53: FutureWarning: `binary_opening` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.opening` instead.
  binary = binary_opening(binary, disk(1))
C:\Users\Goyam Jain\AppData\Local\Temp\ipykernel_9492\554045562.py:56: FutureWarning: Parameter `min_size` is deprecated since version 0.26.0 and will be removed in 2.0.0 (or later). To avoid this warning, please use the parameter `max_size` instead. For more details, see the documentation of `remove_small_objects`. Note that the new threshold removes objects smaller than **or equal to** its value, while the previous parameter only remo

✅ Improved masks saved to: masks_improved
